# Aura SDK - Quickstart

**Agent memory in 3 lines of Python. <1ms recall. No LLM. No cloud.**

Aura gives your AI agent persistent, hierarchical memory that decays, consolidates, and evolves - like human memory.

| | Aura | Others (Mem0, Zep, Cognee) |
|---|---|---|
| LLM required | No | Yes |
| Recall latency | <1ms | 200ms+ / LLM-bound |
| Works offline | Yes | No |
| Binary size | 2.7 MB | Heavy |
| Cost per op | $0 | API billing |

GitHub: [github.com/teolex2020/AuraSDK](https://github.com/teolex2020/AuraSDK) | PyPI: [aura-memory](https://pypi.org/project/aura-memory/)

---

## 1. Install

In [ ]:
!pip install aura-memory -q

## 2. Store & Recall - The Basics

Two methods is all you need: `store()` and `recall()`.

In [ ]:
from aura import Aura, Level

brain = Aura("./demo_brain")

# Store memories at different importance levels
brain.store("User prefers dark mode and Vim keybindings", level=Level.Identity, tags=["preference", "ui"])
brain.store("Rust has zero-cost abstractions and ownership model", level=Level.Domain, tags=["rust", "lang"])
brain.store("Deploy staging before production, always run tests", level=Level.Decisions, tags=["workflow"])
brain.store("Fix login bug - users getting 403 on /api/auth", level=Level.Working, tags=["bug", "auth"])

print("Stored 4 memories across all levels")

In [ ]:
# Recall returns formatted context ready for LLM injection
context = brain.recall("authentication issues", token_budget=2000)
print(context)

In [ ]:
# Structured recall returns scored results with metadata
results = brain.recall_structured("user preferences", top_k=5)
for r in results:
    print(f"  [{r['level']}] score={r['score']:.3f} -- {r['content'][:80]}")

## 3. Memory Levels & Tiers

Aura has 4 levels organized into 2 tiers:

| Tier | Level | Retention | Use Case |
|------|-------|-----------|----------|
| **Cognitive** | Working | 0.80/cycle | Current tasks, recent messages |
| **Cognitive** | Decisions | 0.90/cycle | Choices made, action items |
| **Core** | Domain | 0.95/cycle | Learned facts, domain knowledge |
| **Core** | Identity | 0.99/cycle | User preferences, core identity |

Cognitive tier = fast decay (hours to days). Core tier = slow decay (weeks to months).

In [ ]:
# Query only cognitive tier (recent, ephemeral memories)
recent = brain.recall_cognitive("workflow")
print("=== Cognitive Tier ===")
print(recent)

# Query only core tier (permanent knowledge)
knowledge = brain.recall_core_tier("programming")
print("\n=== Core Tier ===")
print(knowledge)

# Tier statistics
stats = brain.tier_stats()
print(f"\nCognitive: {stats['cognitive_total']}  Core: {stats['core_total']}")

## 4. Trust & Provenance

Not all memories are equal. User input is more trustworthy than web scrapes.

In [ ]:
from aura import TrustConfig

tc = TrustConfig()
tc.source_trust = {"user": 1.0, "api": 0.8, "web_scrape": 0.5}
brain.set_trust_config(tc)

# Store with different trust sources
brain.store("Python 3.13 released October 2024", tags=["python"], channel="user")
brain.store("Python 4.0 coming soon", tags=["python"], channel="web_scrape")

# Trust-weighted recall ranks user-sourced memory higher
results = brain.recall_structured("python release", top_k=5)
for r in results:
    print(f"  score={r['score']:.3f}  trust={r.get('trust', 'N/A')}  -- {r['content']}")

## 5. Namespace Isolation

Keep test data invisible to production recall.

In [ ]:
brain.store("Production config: use PostgreSQL", namespace="prod")
brain.store("Test config: use SQLite", namespace="test")

# Recall only sees default namespace by default
results = brain.recall_structured("database config", top_k=5, namespace="prod")
print("Prod namespace:")
for r in results:
    print(f"  {r['content']}")

# Multi-namespace query
results = brain.recall_structured("database", top_k=5, namespace=["prod", "test"])
print("\nBoth namespaces:")
for r in results:
    print(f"  {r['content']}")

print(f"\nNamespaces: {brain.list_namespaces()}")

## 6. Living Memory - Maintenance

Run a single maintenance cycle: decay, reflect, consolidate, archive.

In [ ]:
report = brain.run_maintenance()
print(f"Decayed: {report.decay.decayed}")
print(f"Promoted: {report.reflect.promoted}")
print(f"Consolidated: {report.consolidation.native_merged}")
print(f"Archived: {report.records_archived}")

## 7. Pluggable Embeddings (Optional)

Aura works without embeddings, but you can add them as a 4th RRF signal.

In [ ]:
# Uncomment to install and use embeddings:
# !pip install sentence-transformers -q

# from sentence_transformers import SentenceTransformer
# model = SentenceTransformer("all-MiniLM-L6-v2")
# brain.set_embedding_fn(lambda text: model.encode(text).tolist())

# Now recall uses 4 signals: SDR + MinHash + Tag Jaccard + Embeddings
# brain.store("Authentication failed for user admin")
# results = brain.recall_structured("login problems", top_k=5)

print("Embeddings are optional -- Aura's 3-signal RRF fusion works great without them.")
print("Uncomment the code above to try the 4th signal boost.")

## 8. Encryption

ChaCha20-Poly1305 with Argon2id key derivation. One argument.

In [ ]:
secure_brain = Aura("./encrypted_demo", password="my-secret-key")
secure_brain.store("Top secret: launch code is 42")
print(f"Encrypted: {secure_brain.is_encrypted()}")

# Data at rest is encrypted -- try reading the raw file and you'll see gibberish
result = secure_brain.recall("launch code")
print(result)
secure_brain.close()

## 9. Performance Benchmark

Let's measure real latency on this machine.

In [ ]:
import time

bench = Aura("./bench_data")

# Store 1000 records
t0 = time.perf_counter()
for i in range(1000):
    bench.store(f"Memory record number {i} with some context about topic {i % 50}", tags=[f"tag{i % 10}"])
store_ms = (time.perf_counter() - t0) * 1000

# Recall benchmark
t0 = time.perf_counter()
for _ in range(100):
    bench.recall_structured("topic context", top_k=10)
recall_ms = (time.perf_counter() - t0) * 1000 / 100

# Search by tag benchmark
t0 = time.perf_counter()
for _ in range(100):
    bench.search(tags=["tag5"])
search_ms = (time.perf_counter() - t0) * 1000 / 100

print(f"Store 1000 records: {store_ms:.1f}ms total ({store_ms/1000:.3f}ms/op)")
print(f"Recall (1K records): {recall_ms:.3f}ms/op")
print(f"Search by tag:       {search_ms:.3f}ms/op")
print(f"\nFor comparison: embedding-based recall = ~200ms+ per call")

bench.close()

## 10. Use with LLMs (Ollama / OpenAI / any)

The core pattern: recall context, inject into system prompt.

In [ ]:
# This is the pattern for ANY LLM integration:

user_message = "What are my UI preferences?"

# 1. Recall relevant context
context = brain.recall(user_message, token_budget=2000)

# 2. Build system prompt with memory
system_prompt = f"""You are a helpful assistant with memory.

{context}

Use the above context to personalize your responses."""

print("System prompt for LLM:")
print("=" * 50)
print(system_prompt)
print("=" * 50)

# 3. Send to your LLM of choice:
# - Ollama: requests.post("http://localhost:11434/api/chat", ...)
# - OpenAI: openai.chat.completions.create(messages=[{"role": "system", "content": system_prompt}])
# - LangChain: ChatPromptTemplate with {context} variable
# - Any HTTP API: just put system_prompt in the system message

print("\nSend this system prompt to any LLM. See examples/ for full integrations.")

## Cleanup

In [ ]:
brain.close()

# Clean up demo data
import shutil
for d in ["./demo_brain", "./encrypted_demo", "./bench_data"]:
    shutil.rmtree(d, ignore_errors=True)

print("Cleaned up. Go build something with Aura!")
print("\nNext steps:")
print("  pip install aura-memory")
print("  GitHub: https://github.com/teolex2020/AuraSDK")
print("  API docs: https://github.com/teolex2020/AuraSDK/blob/main/docs/API.md")
print("  Examples: https://github.com/teolex2020/AuraSDK/tree/main/examples")